<a href="https://colab.research.google.com/github/kgovindaraju123/AIML/blob/master/Mar5GensimwithSpacyWordtoVec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!python -m spacy download en_core_web_lg
import numpy as np

In [0]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint #pretty printer

In [9]:
nytimes=pd.read_csv('nytimes.csv')
nytimes.shape
nytimes.columns


Index(['lead_paragraph', 'datetime', '_id', 'web_url'], dtype='object')

In [0]:
newest_doc=nytimes['lead_paragraph']

In [0]:
nlp=spacy.load("en")
#My list of stop words
stop_list = ["Mrs.","Ms.","say","WASHINGTON","'s","Mr.",]

#Updates SpaCy's default stop_words list with my additional words
nlp.Defaults.stop_words.update(stop_list)

#Iterates over the words in the stop words list and resets the "is_stop" flag
for word in STOP_WORDS:
    lexeme=nlp.vocab[word]
    lexeme.is_stop=True
   

In [0]:
def lemmatizer(doc):
  doc=[token.lemma_ for token in doc if token.lemma_ != '-PRON-']
  doc=u' '.join(doc)
  return nlp.make_doc(doc)

def remove_stopwords(doc):
  doc=[token.text for token in doc if token.is_stop !=True and token.is_punct !=True]
  return doc
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords,name="stopwords",last=True)

In [16]:
doc_list=[]
for doc in tqdm(newest_doc):
  pr=nlp(doc)
  doc_list.append(pr)

In [0]:
doc_list

In [0]:
words=corpora.Dictionary(doc_list)
corpus=[words.doc2bow(doc) for doc in doc_list]


In [0]:
#LDA Latent Dirichlet Allocation Model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=words,num_topics=10,random_state=2,update_every=1,passes=10,alpha='auto',per_word_topics=True)


In [26]:
pprint(lda_model.print_topics(num_words=10))

[(0,
  '0.010*"american" + 0.010*"world" + 0.009*"United" + 0.009*"States" + '
  '0.008*"economic" + 0.008*"year" + 0.007*"country" + 0.007*"America" + '
  '0.006*"business" + 0.006*"economy"'),
 (1,
  '0.020*"President" + 0.012*"tax" + 0.012*"economic" + 0.010*"Reagan" + '
  '0.010*"cut" + 0.009*"House" + 0.009*"Bush" + 0.008*"Congress" + '
  '0.008*"budget" + 0.007*"Obama"'),
 (2,
  '0.019*"market" + 0.013*"financial" + 0.011*"bank" + 0.010*"Street" + '
  '0.010*"company" + 0.010*"Wall" + 0.009*"stock" + 0.007*"investor" + '
  '0.006*"crisis" + 0.006*"year"'),
 (3,
  '0.032*"rate" + 0.023*"Federal" + 0.020*"interest" + 0.019*"Reserve" + '
  '0.012*"market" + 0.011*"week" + 0.011*"Fed" + 0.011*"economy" + '
  '0.010*"inflation" + 0.010*"price"'),
 (4,
  '0.007*"like" + 0.006*"man" + 0.006*"people" + 0.005*"day" + 0.005*"woman" + '
  '0.005*"New" + 0.005*"come" + 0.005*"life" + 0.004*"city" + 0.004*"know"'),
 (5,
  '0.010*"worker" + 0.007*"people" + 0.006*"company" + 0.006*"year" + '
 